## Try lang tong lahat


## Import


In [63]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [64]:
df = pd.read_parquet('parquet/kess.parquet')

#### Drop columns


In [65]:
column_names = df.columns.tolist()
print("Column names:", column_names)

Column names: ['ID', 'Year_Birth', 'Income', 'Kidhome', 'Teenhome', 'Dt_Customer', 'Recency', 'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth', 'Response', 'Complain', 'Total_Children', 'Days since Customer', 'Age_at_Customer_Date', 'Education_Basic', 'Education_Graduation', 'Education_Master', 'Education_PhD', 'Marital_Status_Married', 'Marital_Status_Single', 'Marital_Status_Together', 'Marital_Status_Widow', 'Marital_Status_Married_Kidhome', 'Marital_Status_Married_Teenhome', 'Marital_Status_Single_Kidhome', 'Marital_Status_Single_Teenhome', 'Marital_Status_Together_Kidhome', 'Marital_Status_Together_Teenhome', 'Marital_Status_Widow_Kidhome', 'Marital_Status_Widow_Teenhome', 'Marital_Status_Married_Income', 'Marital_Status_Married_Complain', 'Marital_Status_Single_Income', 'Marital_Status_Single_Complain', 'Marital_Status_Together_

In [66]:
# df.drop(columns=['ID', 'Year_Birth', 'Kidhome',
#         'Teenhome', 'Dt_Customer'], inplace=True)
# df.drop(columns=['ID', 'Year_Birth', 'Total_Children',
#         'Dt_Customer'], inplace=True)
# df.drop(columns=['ID', 'Year_Birth', 'Total_Children',
#         'Dt_Customer', 'Marital_Status', 'Kidhome', 'Teenhome'], inplace=True)
df.drop(columns=['ID', 'Year_Birth', 'Total_Children',
        'Dt_Customer', 'Kidhome', 'Teenhome', 'Income', 'Complain'], inplace=True)
df

,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,...,Marital_Status_Single_Teenhome_Income,Marital_Status_Single_Teenhome_Complain,Marital_Status_Together_Kidhome_Income,Marital_Status_Together_Kidhome_Complain,Marital_Status_Together_Teenhome_Income,Marital_Status_Together_Teenhome_Complain,Marital_Status_Widow_Kidhome_Income,Marital_Status_Widow_Kidhome_Complain,Marital_Status_Widow_Teenhome_Income,Marital_Status_Widow_Teenhome_Complain
0,0,189,104,379,111,189,218,1,4,4,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
1,0,464,5,64,7,0,37,1,7,3,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
2,0,134,11,59,15,2,30,1,3,2,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
3,0,10,0,1,0,0,0,1,1,0,...,0.0,0,32474.0,0,32474.0,0,0.0,0,0.0,0
4,0,6,16,24,11,0,34,2,3,1,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2196,99,372,18,126,47,48,78,2,5,2,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
2197,99,5,10,13,3,8,16,1,1,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
2198,99,185,2,88,15,5,14,2,6,1,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
2199,99,267,38,701,149,165,63,1,5,4,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0


## Baseline Modelling


In [67]:
X = df.drop('Response', axis=1).iloc[:, 1:]
y = df['Response']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y)

In [68]:
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)

# clf = LogisticRegression(random_state=0).fit(X_train, y_train)
# preds = clf.predict(scaler.transform(X_test))

# acc = accuracy_score(y_test, preds)
# prec = precision_score(y_test, preds)
# rec = recall_score(y_test, preds)
# f1 = f1_score(y_test, preds)
# auc = roc_auc_score(y_test, preds)

In [69]:
# print("Accuracy: %.4f" % acc)
# print("Precision: %.4f" % prec)
# print("Recall: %.4f" % rec)
# print("F1: %.4f" % f1)
# print("AUC: %.4f" % auc)

In [70]:
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

In [71]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

classifiers = {
    'Logistic Regression': LogisticRegression(random_state=0),
    'SVM': SVC(random_state=0, probability=True),
    'Naive Bayes': GaussianNB(),
    'Decision Tree': DecisionTreeClassifier(random_state=0),
    'K-Nearest Neighbors': KNeighborsClassifier()
}

scores = {name: {} for name in classifiers.keys()}

for name, clf in classifiers.items():
    clf.fit(X_train_scaled, y_train)
    preds = clf.predict(X_test_scaled)

    scores[name]['Accuracy'] = accuracy_score(y_test, preds)
    scores[name]['Precision'] = precision_score(y_test, preds)
    scores[name]['Recall'] = recall_score(y_test, preds)
    scores[name]['F1 Score'] = f1_score(y_test, preds)

    if hasattr(clf, "predict_proba"):
        pred_probs = clf.predict_proba(X_test_scaled)[:, 1]
        scores[name]['ROC AUC'] = roc_auc_score(y_test, pred_probs)
    else:
        scores[name]['ROC AUC'] = 'N/A'

scores_df = pd.DataFrame(scores).T
scores_df

,Accuracy,Precision,Recall,F1 Score,ROC AUC
Logistic Regression,0.848714,0.491525,0.292929,0.367089,0.784823
SVM,0.853253,0.555556,0.101010,0.170940,0.788436
Naive Bayes,0.158850,0.149002,0.979798,0.258667,0.506740
Decision Tree,0.835098,0.450980,0.464646,0.457711,0.682025
K-Nearest Neighbors,0.838124,0.394737,0.151515,0.218978,0.659450


## Small Findings


Models result with better scores when you have columns kidhome and teenhome rather than total children
